In [1]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import argparse

/Users/shankar/envs/automate/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
IMG_PATH = ''

In [3]:
# Grab the mnist dataset, if this is your first time running this script , 
# download may take a minute -- the 55mb mnist dataset will be downloaded
dataset = datasets.fetch_mldata("MNIST Original")

In [4]:
# Scale the raw pixel intensities to the range [0, 1.0], then construct 
# the training and testing splits
data = dataset.data.astype("float")/255.0
(trainX, testX, trainY, testY) = train_test_split(data,
    dataset.target, test_size=0.25)

In [5]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(52500, 784)
(52500,)
(17500, 784)
(17500,)


In [6]:
testY[:10]

array([1., 0., 5., 0., 9., 7., 9., 9., 6., 3.])

In [7]:
# Conver the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [8]:
trainY[:10]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [9]:
# Define the 784-256-128-10 architecture using Keras
model = Sequential()
model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model using SGD
print("[INFO] training network...")
sgd = SGD(0.01)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=128)

[INFO] training network...
Train on 52500 samples, validate on 17500 samples
Epoch 1/100
52500/52500 [==============================] - 2s 38us/step - loss: 2.2874 - acc: 0.1708 - val_loss: 2.2491 - val_acc: 0.2521
Epoch 2/100
52500/52500 [==============================] - 2s 33us/step - loss: 2.2190 - acc: 0.3377 - val_loss: 2.1877 - val_acc: 0.4297
Epoch 3/100
52500/52500 [==============================] - 2s 33us/step - loss: 2.1495 - acc: 0.5011 - val_loss: 2.1067 - val_acc: 0.5227
Epoch 4/100
52500/52500 [==============================] - 2s 34us/step - loss: 2.0530 - acc: 0.5767 - val_loss: 1.9904 - val_acc: 0.6539
Epoch 5/100
52500/52500 [==============================] - 2s 34us/step - loss: 1.9180 - acc: 0.6303 - val_loss: 1.8330 - val_acc: 0.6239
Epoch 6/100
52500/52500 [==============================] - 2s 36us/step - loss: 1.7424 - acc: 0.6514 - val_loss: 1.6409 - val_acc: 0.6780
Epoch 7/100
52500/52500 [==============================] - 2s 35us/step - loss: 1.5463 - acc: 0

In [ ]:
# Evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in lb.classes_])

In [ ]:
# Plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])